In [ ]:
# Problem is at conjugate gradient
# make sure my torch conjugate gradient and the openai's numpy conjugate gradient algo are 
# correct

In [80]:
from cg import cg, _cg
import numpy as np

import torch

A = np.random.rand(5, 5)
b = np.random.rand(5)

def linear_fn(x):
    return A@x

a = _cg(linear_fn, b, cg_iters=10)

A = torch.from_numpy(A)
b = torch.from_numpy(b)

a_ = cg(linear_fn, b, cg_iters=10).numpy()

print(np.allclose(a, a_))

True


In [96]:
# check if my implementation of FVP can correctly compute the 
# Av
import models
import cg
import imp

imp.reload(models)
imp.reload(cg)
import utils as U
from models import MLPTest
from cg import cg


input_shape=4
model = MLPTest(input_shape)
cg_damping = 1e-2

ob = torch.rand(12, input_shape)
kl = model(ob).mean()


def compute_fvp(flat_tangent, ob):
    dkl_params = U.flatten(
        torch.autograd.grad(kl, model.parameters(), 
                            retain_graph=True,
                           create_graph=True))
    
    z = torch.dot(dkl_params, flat_tangent)
    return U.flatten(torch.autograd.grad(z, model.parameters(), 
                                         retain_graph=True))

def fisher_vector_product(p):
    return compute_fvp(p, ob)

g = U.flatten(torch.autograd.grad(kl, model.parameters(), retain_graph=True))
print('sum of gradient', g.sum())

x=cg(fisher_vector_product, g, cg_iters=10, verbose=True)

sum of gradient tensor(1.9902)
      iter residual norm  soln norm
         0       12.5          0
         1   1.16e+04         46
         2       1.47       1.51
         3       53.5       4.72
         4      0.633       1.66
         5       15.7       3.39
         6      0.277       1.73
         7       3.51       2.26
         8     0.0947       1.76
         9      0.606       1.86
        10     0.0438       1.77


In [112]:
x, y,  *_ = np.arange(10)
y

1

In [107]:
np.array(x)

array([[1, 2]])

In [1]:
import torch

def mycg(f_Ax, b, cg_iters=10, verbose=False, residual_tol=1e-10):
    x = torch.zeros_like(b, dtype=torch.float32)
    
    # residual and A-conjugate basis
    r, p = b.clone(), b.clone()
    rr = torch.dot(r, r)
    
    fmtstr =  "%10i %10.3g %10.3g"
    titlestr =  "%10s %10s %10s"
    if verbose: print(titlestr % ("iter", "residual norm", "soln norm"))
        
    for i in range(cg_iters):
        if verbose: print(fmtstr % (i, rr, np.linalg.norm(x.numpy())))
        Ap = f_Ax(p)
        assert p.shape == Ap.shape
        alpha = rr / torch.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rr_new = torch.dot(r, r)
        beta = - rr_new / rr
        p = r - beta * p
        rr = rr_new
        if rr < residual_tol:
            print("residual tolerance achieved, good enough for now")
            break
    
    if verbose: print(fmtstr % (i+1, rr, np.linalg.norm(x.numpy())))
    return x
        
def mycg_numpy(f_Ax, b, cg_iters=10, verbose=False, residual_tol=1e-10):
    x = np.zeros_like(b, dtype=np.float32)
    
    # residual and A-conjugate basis
    r, p = b.copy(), b.copy()
    rr = np.dot(r, r)
    
    fmtstr =  "%10i %10.3g %10.3g"
    titlestr =  "%10s %10s %10s"
    if verbose: print(titlestr % ("iter", "residual norm", "soln norm"))
        
    for i in range(cg_iters):
        if verbose: print(fmtstr % (i, rr, np.linalg.norm(x)))
        Ap = f_Ax(p)
        assert p.shape == Ap.shape
        alpha = rr / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rr_new = np.dot(r, r)
        beta = - rr_new / rr
        p = r - beta * p
        rr = rr_new
        if rr < residual_tol:
            print("residual tolerance achieved, good enough for now")
            break
    
    if verbose: print(fmtstr % (i+1, rr, np.linalg.norm(x)))
    return x
        


n = 36

# Generate a positve definite
# symmetric matrix
A = np.random.rand(n, n)
A = A@A.T/1e9

print('sum of A', np.sum(A))

g = np.random.rand(n)

def linear_fn(p):
    return np.dot(A, p)



x = mycg_numpy(linear_fn, g, cg_iters=15, verbose=True)

NameError: name 'np' is not defined

I roughly know where the problem is, look at the surrgain from my implementation
It is on the scale of 10^2
```
********** Iteration 1 ************
sampling
done in 0.395 seconds
XXXXX atarg shape torch.Size([1024])
mean atarg tensor(-2.2631e-07)
computegrad
done in 0.003 seconds
cg
sum of g tensor(8.5350)
      iter residual norm  soln norm
         0   1.34e+03          0
         1        204       13.2
         2       35.5       16.5
         3       10.9       41.2
         4       4.07       44.9
         5     0.0867       73.3
         6       1.99       75.3
         7     0.0149       75.9
         8      0.031       75.9
         9   3.12e-05       76.9
        10   0.000249       76.9
done in 0.030 seconds
Expected: 1.301 Actual: 0.705
stepsize ok
vf
done in 0.002 seconds
---------------------------------------
| entloss         | tensor(0.)        |
| entropy         | tensor(1.4030)    |
| EpisodesSoFar   | 0                 |
| EpLenMean       | 200               |
| EpRewMean       | -1.27e+03         |
| EpThisIter      | 5                 |
| ev_tdlam_before | 2.38e-07          |
| meankl          | tensor(0.0010)    |
| optimgain       | tensor(-356.8666) |
| surrgain        | tensor(-356.8666) |
| TimeElapsed     | 0.97              |
| TimestepsSoFar  | 2000              |
---------------------------------------
```

OpenAi's implementation is 

```----------------------------------
| entloss         | 0.0          |
| entropy         | 1.4265722    |
| EpisodesSoFar   | 5            |
| EpLenMean       | 200          |
| EpRewMean       | -1317.9513   |
| EpThisIter      | 5            |
| ev_tdlam_before | -0.000287    |
| meankl          | 0.0007376976 |
| optimgain       | 0.0027435264 |
| surrgain        | 0.0027435264 |
| TimeElapsed     | 2.19         |
| TimestepsSoFar  | 1000         |
----------------------------------

```

Their surrgain is confined in much smaller scale. I need to figure out why that's the case. 
When I removed advtange from the surrgain computation, my implmentaiton and openai agree. 

Our atarg (advantage) is on the same scale after normalize.